In [1]:
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [2]:
Train = pd.read_pickle('data/crabTrain')
Test = pd.read_pickle('data/crabTest')

/home/msackel/.conda/envs/msackel/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-06 12:29:07.071723. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [4]:
loopParameters = {
    'max_depth':[4,5,6],
    'learning_rate':[10,1,0.1,0.01],
    'gamma':[10,1,0.1,0.01],
}
parameters = {
    'booster':'gbtree',
    'silent':0, 
    'nthread':20 ,
    'subsample': 0.3,
    'objective':'binary:logistic',         
    'eval_metric':'auc',
}
rnd = XGBClassifier(**parameters)
rnd = XGBClassifier(**parameters)

/home/msackel/.conda/envs/msackel/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-06 12:29:12.930417. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [5]:
clf = GridSearchCV(rnd, param_grid=loopParameters, cv=2, scoring='roc_auc')
clf.fit(Train.drop('label', axis=1), Train.label)

/home/msackel/.conda/envs/msackel/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-06 12:29:14.401668. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


GridSearchCV(cv=2, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=20, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=0, subsample=0.3),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [4, 5, 6], 'learning_rate': [10, 1, 0.1, 0.01], 'gamma': [10, 1, 0.1, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [6]:
bestXGBoost = pd.DataFrame(clf.cv_results_)
bestXGBoost.to_pickle('bestXGBoost')

In [7]:
bestXGBoost = pd.read_pickle('bestXGBoost')
bestXGBoost[bestXGBoost['rank_test_score']==[1]]

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_gamma,param_learning_rate,param_max_depth,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
5,2.301135,0.121375,0.908006,0.925871,10,1,6,"{'gamma': 10, 'learning_rate': 1, 'max_depth': 6}",1,0.901934,0.926287,0.914078,0.925454,0.043617,0.00243,0.006072,0.000416


In [8]:
bestXGBoost[bestXGBoost['rank_test_score']==[2]]

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_gamma,param_learning_rate,param_max_depth,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
4,2.001193,0.11889,0.907449,0.922298,10,1,5,"{'gamma': 10, 'learning_rate': 1, 'max_depth': 5}",2,0.899844,0.92125,0.915054,0.923347,0.046023,0.00476,0.007605,0.001048


In [11]:
parameters = {
    'max_depth':5,
    'learning_rate':1,
    'gamma':10,
    'booster':'gbtree',
    'silent':0, 
    'nthread':20 ,
    'subsample': 0.3,
    'objective':'binary:logistic',         
    'eval_metric':'auc',
}
rnd = XGBClassifier(**parameters)
rnd.fit(Train.drop('label', axis=1), Train.label)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='auc', gamma=10, learning_rate=1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=20, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=0, subsample=0.3)

In [12]:
pre = rnd.predict_proba(Test.drop('label', axis=1))[:,1]

In [13]:
roc_auc_score(Test.label, pre)

0.92728488388888886